In [1]:
import pandas as pd
import json
import numpy as np

In [2]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

Import data.gov.ro etalon data

In [9]:
megye={'Fehér':'AB', 'Arad':'AR', 'Bukarest':'B', 'Bákó':'BC', 'Bihar':'BH', 'Beszterce-Naszód':'BN',
       'Brassó':'BV', 'Kolozs':'CJ', 'Kovászna':'CV', 'Krassó-Szörény':'CS', 'Hunyad':'HD',
       'Hargita':'HR', 'Máramaros':'MM', 'Maros':'MS', 'Szeben':'SB', 'Szatmár':'SM', 'Szilágy':'SJ',
       'Temes':'TM'}

In [3]:
ro={'Ă':'A','Ş':'S','Â':'A','Ș':'S','Ț':'T','Â':'A','Î':'I','Ă':'A','Ţ':'T','-':' ','SC.GEN.':'','I VIII':''}
def roman(s):
    return replacer(s,ro)

In [4]:
gov=pd.read_excel('data/raw/data.gov.ro/datedeschise-retea-2019-2020.xls')

In [5]:
locs=gov.set_index('Judet')['Localitate']

In [15]:
mgov_clean=json.loads(open('data/clean/mgov_clean.json','r').read())

Load processed diak data

In [10]:
dg=pd.read_excel('data/clean/erdely5_manual.xlsx').drop('Unnamed: 0',axis=1)

In [55]:
m='Arad'

In [58]:
loc_datas=[]
for m in megye:
    print(m)
    loc_data=pd.read_excel('data/clean/guessed/'+m+'.xlsx').drop('Unnamed: 0',axis=1)
    gov_loc_ids=[]
    for i in loc_data.T.iteritems():
        nev=i[1]['guessed_names2']
        clean_nev=i[1]['clean_names']
        telepules=i[1]['telepules']
        telepules=i[1]['telepules']
        if nev in mgov_clean[m][telepules]:
            gov_loc_ids.append(mgov_clean[m][telepules][nev])
        elif 'nferred' in mgov_clean[m][telepules][clean_nev]:
            for i in [item for sublist in [list(mgov_clean[m].keys()) for t in mgov_clean[m]] for item in sublist]:
                if nev in mgov_clean[m][i]:
                    gov_loc_ids.append(mgov_clean[m][i][nev])
                    break
        else:
            gov_loc_ids.append('')
    loc_data['gov_loc_ids']=gov_loc_ids
    loc_data['Megye']=m
    loc_datas.append(loc_data)
loc_datas=pd.concat(loc_datas)

Fehér
Arad
Bukarest
Bákó
Bihar
Beszterce-Naszód
Brassó
Kolozs
Kovászna
Krassó-Szörény
Hunyad
Hargita
Máramaros
Maros
Szeben
Szatmár
Szilágy
Temes


In [60]:
guessed_locs=dg.set_index(['Megye','telepules','Név']).join(loc_datas.set_index(['Megye','telepules','Név'])).reset_index().set_index('gov_loc_ids')

In [65]:
data=guessed_locs.join(gov)

In [66]:
data.to_excel('data/clean/erdely6.xlsx')